# Ayudantía 3 - Sistemas Recomendadores: Pyreclab

**Nombre alumno(s):**

## Setup

**Paso 1:** Descarga de archivos:

*   `dictionary.p`
*   `dictionary-stemm.p`
*  `tfidf_model.p`
*  `tfidf_model-stemm.p`

In [ ]:
# Descargue los archivos ejecutando este comando
!curl -L -o 'resources.tar.gz' 'https://github.com/PUC-RecSys-Class/Syllabus/blob/master/Practico%203/files/resources.tar.gz'


# Descomprima el archivo
!tar -xvf resources.tar.gz

**Paso 1.5:** Descarga del dataset:

In [ ]:
# Puede descargar el dataset ejecutando el siguiente comando
!curl -L -o 'dataset.tar.gz' 'https://github.com/PUC-RecSys-Class/Syllabus/blob/master/Practico%203/files/dataset.tar.gz'

# Y descomprimirlo con
!tar -xvf dataset.tar.gz

**Paso 2:** Para este práctico es necesario instalar las siguentes dependencias:

In [ ]:
!pip install nltk
!pip install sklearn
!pip install gensim
!pip install pandas
!pip install numpy

In [ ]:
import os
import nltk
import sklearn
import gensim
import string
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models, similarities
from sklearn.neighbors import NearestNeighbors

## Preprocesamiento de datos

Lo primero es descargar las librerías de NLTK necesarias:

In [ ]:
# Download corpora
nltk.download('punkt')

Para comenzar cargaremos el set de datos en un *dataframe* de Pandas, e imprimimos los 5 primeros registros para visualizar la estructura de los datos.

In [ ]:
corpus_df = pd.read_csv('./corpus1.csv', sep='\t',
                        header=None, encoding='latin')
corpus_df.columns = ['id', 'title', 'abstract']
corpus_df = corpus_df[['id', 'title', 'abstract']]
corpus_df[:5]

Lo siguiente es implementar una función que transforme texto no estructurado a una lista de tokens procesados.

In [ ]:
stemm = False
stemmer = PorterStemmer()


def get_tokens(text):
    lowers = text.lower()
    no_punctuation = lowers.translate(
        {ord(c): None for c in string.punctuation})
    tokens = nltk.word_tokenize(no_punctuation)
    if stemm:
        tokens = map(stemmer.stem, tokens)

    return tokens


get_tokens("I'm a super student for recommender systems!")

**Pregunta:** Explique en sus palabras qué hace la función `get_tokens()`.

**Respuesta:**



Ahora se tiene que generar un diccionario con todas las palabras del *corpus*.

Se recomienda revisar la documentación de gensim y leer cómo usar los diccionarios: [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html)

In [ ]:
dict_file = './resources/dictionary-stemm.p' if stemm else './resources/dictionary.p'
if os.path.isfile(dict_file):
    dictionary = corpora.dictionary.Dictionary().load(dict_file)
else:
    dictionary = corpora.dictionary.Dictionary(
        documents=corpus_df.tokenised_abstract.tolist())
    dictionary.save(dict_file)

corpus_df['tokenized_abstract'] = corpus_df.abstract.map(get_tokens)
corpus_df[:5]

**Pregunta:** Explique a qué corresponde la columna `tokenised_abstract` del dataframe.

**Respuesta:**

`



In [ ]:
corpus_df['bow'] = corpus_df.tokenized_abstract.map(dictionary.doc2bow)
del corpus_df['tokenized_abstract']
corpus = corpus_df['bow'].tolist()
corpus_df[:5]

**Pregunta:** Explique a qué corresponde la columna `bow`

**Respuesta:**


## Tf-idf

In [ ]:
tfidf_model_file = 'resources/tfidf_model-stemm.p' if stemm else 'resources/tfidf_model.p'
if os.path.isfile(tfidf_model_file):
    tfidf_model = models.tfidfmodel.TfidfModel().load(tfidf_model_file)
else:
    tfidf_model = models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary)
    tfidf_model.save(tfidf_model_file)

corpus_df['tf_idf'] = tfidf_model[corpus_df.bow.tolist()]
corpus_df[:5]

**Pregunta:** Explicar a qué corresponde la columna `tf_idf` y por qué es útil en el procesamiento de texto. Mencione sus 2 principales partes, mediante la explicación del puntaje.

**Respuesta:**

## Generar recomendaciones

En esta sección se implementan las funciones necesarias para poder generar recomendaciones dado lo que un usuario ha consumido. De manera artificial, se samplearán 3 documentos aleatorios que representarán al usuario objetivo (`sample`). Luego tendrás que generar diferentes recomendaciones y evaluar los resultados.

In [ ]:
# Random user

samples = corpus_df.sample(3)

for n, (ix, paper) in enumerate(samples.iterrows()):
    idx, title, abstract, bow, tf_idf = paper
    print('%d) %s' % (n+1, title))
    print('')
    print(abstract)
    print('\n')

In [ ]:
# Recommendation functions

N = len(dictionary)


def to_sparse(matrix):
    return csr_matrix([gensim.matutils.sparse2full(row, length=N) for row in matrix])


def make_recommendations(model, metric, neighbors):
    M = len(corpus)

    X = to_sparse(corpus_df[model].tolist())
    document_index = NearestNeighbors(n_neighbors=(
        neighbors + 1), algorithm='brute', metric=metric).fit(X)
    return document_index


def print_recommendations(indexes, model):
    for n, (ix, paper) in enumerate(samples.iterrows()):
        dists, neighbors = indexes.kneighbors(
            [gensim.matutils.sparse2full(paper[model], length=N)])
        print(paper['title'])
        print('')
        print('Documentos cercanos: ')
        i = 1
        for neighbour in neighbors[0]:
            if ix != neighbour:
                line = str(i) + ". " + corpus_df.iloc[neighbour]['title']
                print(line)
                i += 1
        print('\n')

A continuación deberá utilizar las funciones implementadas anteriormente para generar nuevas recomendaciones variando los parámetros del modelo. Agregue nuevas celdas para cada implementación y/o pregunta.


** Pregunta:** Ejecute el modelo utilizando como representación tf-idf y una métrica de distancia euclideana. Modifique el parámetro nearest_neighbors a [5, 10, 20]. ¿qué efecto tiene el modelo en las recomendaciones observadas?

**Respuesta:**

**Pregunta:** Eligiendo un valor fijo para *nearest neighbors* y utilizando representación tf-idf, ejecute el modelo con métrica de distancia *cosine*.¿Qué efecto tiene la métrica de distancia en las recomendaciones observadas?

**Respuesta:**


In [ ]:
# Recommendation example

doc_idx = make_recommendations('tf_idf', 'euclidean', 5)
print_recommendations(doc_idx, 'tf_idf')